# Analysis of data from the Covid-19 pandemic

In [ ]:
%reset -f
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
from scipy.integrate import odeint
import scipy.optimize as op
from datetime import datetime, timedelta
from copy import deepcopy
from collections import namedtuple
from pathlib import Path

In [ ]:
MPL_FIG_TITLE_DATE_FORMAT = '%b%d'
MPL_FIG_SIZE_LARGE =(12.1, 8)
PD_FIG_SIZE_LARGE = (15, 8)
PD_FIG_SIZE_SMALL = (10, 4)
plt.rcParams['figure.figsize'] = PD_FIG_SIZE_LARGE
pd.options.display.max_rows = 8

UNIX_DATE_DELTA = datetime(1970, 1, 1)
# numpy datetime64 timestamp is represented in nano seconds
DATETIME64_TO_DATETIME_SCALER = 1e-9
sns.set()
FIG_COLOR_PALETTE = sns.color_palette()

# https://de.wikipedia.org/wiki/Deutschland
GERMANY_INHABITANTS = 83e6
GERMANY_INTENSIVE_CARE_PLACES = 23e3
GERMANY_LOCKDOWN_DATE = datetime(2020, 3, 23)
BAVARIA_LOCKDOWN_DATE = datetime(2020, 3, 31)
GERMANY_LOCKDOWN_WEAKEN_1 = datetime(2020, 4, 20)

isodate2xtick = lambda date: (date - UNIX_DATE_DELTA).days

def markdown_table_fmt(params):
    f = lambda x: ("%.4g" % x if isinstance(x, float) else
                   x.isoformat()[:10] if isinstance(x, datetime) else
                   x)
    s = (f(p) for p in params)
    return "| " + " | ".join(s) + " |"

In [ ]:
%%html
<!-- left align markdown tables -->
<style>table {float:left;}</style>

## Loading the Covid-19 cases data

Load the `cases.csv`, which was generated by `retriev_clean_data.ipynb`.

In [ ]:
filename_cases = Path('cases.csv')
assert filename_cases.exists()

df = pd.read_csv(filename_cases, index_col=[0], parse_dates=[0])
assert df.index.dtype == np.dtype('M8[ns]')

df.tail(3)

In [ ]:
# initialize df dependent constants
GERMANY_LOCKDOWN_INDEX = np.where(df.index == GERMANY_LOCKDOWN_DATE)[0][0]
GERMANY_LOCKDOWN_INDEX

### Plotting Covid-19 cases data

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=True,
                       figsize=MPL_FIG_SIZE_LARGE, constrained_layout=True)

for i, key in enumerate(df.keys()):
    a_idx = 0 if 'JHU' in key else 1
    df[[key]].diff().plot.bar(ax=ax[a_idx], alpha=0.3, align='edge', color=FIG_COLOR_PALETTE[i], width=1, stacked=False)
    ax[a_idx].set_ylim(None, 7000)
labels = (l.get_text()[:10] if i % 2 else "" for i, l in enumerate(ax[1].get_xticklabels()))
ax[1].set_xticklabels(labels, rotation=60);

In [ ]:
ax = df.plot(style='o-', figsize=PD_FIG_SIZE_LARGE, logy=True, alpha=0.5)

## Fit $2^x$ to the JHU data of Germany

#### TODO:
- [ ] can we get an estimate of the reproduction rate  $R_0$ also from the exponentail rate?

### Scratchpad for getting intuition on exponentail functions

In [ ]:
x = np.arange(4)
np.exp(x), 2.718*2.718*2.718

In [ ]:
np.exp(1)**(x), np.exp(x)

In [ ]:
np.log(2**6)/np.log(2), 2**6

### Fit $2^x$ for certain time spans

In [ ]:
EXP_FIT_COUNTRY_KEY = "Germany_JHU"

def plot_pow2_fit(df, begin_date, end_date, expfct,
                  ndays=None, legend=False, figsize=PD_FIG_SIZE_SMALL, linestyle='o--',
                  date_to_xtick_fct=isodate2xtick):
    """
    param date_to_xtick_fct: one of {mdates.date2num, isodate2xtick}
    """
    # plot the dateframe
    mask = (begin_date <= df.index) & (df.index <= end_date)
    ax = df.loc[mask].plot(style='o-', figsize=figsize, legend=False)
    # calculate ndays
    _ylim = ax.get_ylim()
    x1 = date_to_xtick_fct(begin_date)
    if ndays is None:
        _x1, x2 = ax.get_xlim()
        assert abs(_x1 - x1) < 0.5, (
            'begin date repr differs: _x1 = %d, x1 = %d \n'
            'choose the other date_to_xtick_fct = {mdates.date2num, isodate2xtick}' % (_x1, x1)
        )
        arange_n = np.arange(int(x2 - x1 + 1))
    else:
        arange_n = np.arange(ndays)
    # plot the estimated exponentail function
    ax.plot(x1 + arange_n, expfct(arange_n), linestyle)
    ax.set_title("%s - %s" % ((begin_date.strftime(MPL_FIG_TITLE_DATE_FORMAT), end_date.strftime(MPL_FIG_TITLE_DATE_FORMAT))))
    ax.set_ylim(*_ylim)
    ax.set_xlim(x1, date_to_xtick_fct(end_date))
    ax.set_ylabel('total cases')
    return ax

def calc_pow2_zero(df, begin_date, rate, country=EXP_FIT_COUNTRY_KEY):
    y0 = df.loc[begin_date == df.index, country][0]
    x0 = _rate * np.log(y0) / np.log(2)
    return x0

In [ ]:
_begin_date = datetime(2020, 4, 10)
_end_date = datetime(2020, 4, 19)
_rate = 35.5

_x0 = calc_pow2_zero(df[[EXP_FIT_COUNTRY_KEY]], _begin_date, _rate)
expfct = lambda day_range: 2**((day_range + _x0)/_rate)
ax = plot_pow2_fit(df[[EXP_FIT_COUNTRY_KEY]], _begin_date, end_date=_end_date, expfct=expfct)

In [ ]:
_begin_date = datetime(2020, 4, 19)
_end_date = datetime.now() + timedelta(days=1)
_rate = 53.5

_x0 = calc_pow2_zero(df[[EXP_FIT_COUNTRY_KEY]], _begin_date, _rate)
expfct = lambda day_range: 2**((day_range + _x0)/_rate)
ax = plot_pow2_fit(df[[EXP_FIT_COUNTRY_KEY]], _begin_date, end_date=_end_date, expfct=expfct)

EXP_RATE = _rate

### $2^x$ manually fitted, with $x \, \hat{=} \, \frac{\text{days}}{\text{rate}}$
Time spans where identified by looking at the log plot above

| begin date |  end date  | rate |
| ---------- | ---------- | ----:|
| 2020-02-25 | 2020-03-11 |  2.2 |
| 2020-03-13 | 2020-03-19 |  2.9 |
| 2020-03-21 | 2020-03-28 |  5.1 |
| 2020-03-28 | 2020-04-03 |  9.1 |
| 2020-04-03 | 2020-04-04 | 13.2 |
| 2020-04-03 | 2020-04-09 | 16.1 |
| 2020-04-10 | 2020-04-19 | 35.5 |
| 2020-04-19 | 2020-04-?? | 53.5 |

In [ ]:
params = [_begin_date, _end_date, _rate]     
print(markdown_table_fmt(params))

### Estimate intensive care places based on extrapolated expenential function with last `RATE`

The idea of this plot is to relate a pure exponentail growth with the number of intensive care places ($\text{icp} \approx 23e3$ for Germany). Assuming 1% of the infected people need intensive care and that people are 10 days in intensive care, results in the threshold $\frac{icp}{0.01 \cdot 10} = 230e3$.

In [ ]:
_begin_date = datetime(2020, 4, 10)
_end_date = datetime(2021, 5, 31)
_ndays = (_end_date - _begin_date).days

_x0 = calc_pow2_zero(df[[EXP_FIT_COUNTRY_KEY]], _begin_date, _rate)
diffexpfct = lambda day_range: np.diff(2**((np.concatenate([[0], day_range]) + _x0)/EXP_RATE))
ax = plot_pow2_fit(df[[EXP_FIT_COUNTRY_KEY]].diff(), _begin_date, end_date=_end_date,
                   expfct=diffexpfct, ndays=_ndays, figsize=PD_FIG_SIZE_SMALL, linestyle='--')
ax.set_ylabel('new cases per day')

_kwargs = dict(linestyle='--', linewidth=2, zorder=0)
ax.axhline(GERMANY_INTENSIVE_CARE_PLACES, color='0.8', **_kwargs)
ax.grid(color='0.9')
ax.axhline(GERMANY_INTENSIVE_CARE_PLACES * 10, color='r', **_kwargs)
ax.set_ylim(0, GERMANY_INTENSIVE_CARE_PLACES * 10 * 1.1)

'exp basis, for z**(day_range + _x0), z =', 2**(1/EXP_RATE)

## Fit sigmoid function to the JHU data of Germany

The simoid function is the deriveate of the logistic function.

#### Logistic function primer

Based on https://mlnoga.github.io/covid19-analysis there are three differing states in the evolution of Covid-19 pandemic for each country, represented by the
* exponential function, the
* logistic function and the 
* logistic function with a long tail.

Here we look into the logistic function, as defined in https://de.wikipedia.org/wiki/Logistische_Funktion as follows:

    Die logistische Funktion, wie sie sich aus der diskreten logistischen Gleichung ergibt, beschreibt den Zusammenhang zwischen der verstreichenden Zeit und einem Wachstum, beispielsweise einer idealen Bakterien­population. Hierzu wird das Modell des exponentiellen Wachstums modifiziert durch eine sich mit dem Wachstum verbrauchende Ressource – die Idee dahinter ist also etwa ein Bakteriennährboden begrenzter Größe. Zur Anfangszeit ist der Funktionswert nicht 0, sondern es gilt f(0)>0.

$f(t) = G \frac{1}{1 + e^{-kGt + c}} $

$f'(t) = k f(t) (G - f(t))$

We introduc $b = k G$ for numerical stability, as G is large ($G \approx 1.4e5$) and k is small ($k \approx 1e-6$).

#### TODO:
- [ ] plot covariance
- [ ] describe variables of equations above

In [ ]:
%reset_selective -f ([xy][dip][A-z]+[0-9]*)
#%whos

In [ ]:
LOGIT_FIT_COUNTRY_KEY = 'Germany_JHU'
LOGIT_FIT_START_DATE = datetime(2020, 3, 2)   # repr idx 40

mask = df.index >= LOGIT_FIT_START_DATE
yintegrated = df.loc[mask, LOGIT_FIT_COUNTRY_KEY].values.astype('f8')

LogitFitParams = namedtuple('LogitFitParams', 'G b c'.split())

def func(t, G, b, c):
    return G / (1.0 + np.exp(-b * t + c))

def func_derivative(t, G, b, c):
    _f = func(t, G, b, c)
    return b * _f * (1 - _f / G)

# we fit the number of new infections, which is the derivative
# of the confirmed total cases
ydata = np.diff(yintegrated)
xdata = np.arange(1, len(ydata) + 1, dtype='f8')

p0 = LogitFitParams(G=ydata.max(), b=0.1 , c=10)
_popt, pcov = op.curve_fit(func_derivative, xdata, ydata, p0)
popt = LogitFitParams(*_popt)
popt, pcov

In [ ]:
logit_days_to_predict = 30
xpredict = np.arange(len(xdata) + logit_days_to_predict)
ypredict = func_derivative(xpredict, *popt)

fig, ax = plt.subplots(2, 1, sharex=True, figsize=MPL_FIG_SIZE_LARGE)
ax[0].scatter(xdata, ydata, label='# new cases of %s' % LOGIT_FIT_COUNTRY_KEY)
ax[0].plot(xpredict, ypredict, 'g--', label='# predicted new cases of %s' % LOGIT_FIT_COUNTRY_KEY)
ax[1].scatter(xdata, yintegrated[1:])
ax[1].plot(xpredict, func(xpredict, *popt), 'g--')

IDX_OFFSET_TODAY = len(ydata)
idx100 = np.argmin(abs(ypredict[IDX_OFFSET_TODAY:] - 100)) + IDX_OFFSET_TODAY
date100 = LOGIT_FIT_START_DATE + timedelta(days=int(idx100))
ax[0].axvline(idx100, color='0.6', ls='--', label='100 new cases @ %s' % date100.strftime(MPL_FIG_TITLE_DATE_FORMAT))
ax[0].legend().get_frame().set_alpha(0.9)
ax[1].set_xlabel('time [days]')
for i in range(2):
    ax[i].set_ylabel('confirmed cases')

for i in [-3, -2, -1]:
    _scale = 3.5 / 10
    alpha = 1 * (1 + _scale * (i + 1))
    print(i, ("%.2f" % alpha, ydata[i]), end=' ')
    ax[0].text(xdata[i] * 1.01, ydata[i], "%d" % ydata[i], alpha=alpha)

fig.tight_layout()

### Date of 100 new infections Germany based on sigmoid fit

| Date of fit | Date of 100 new infections |        G | b = k * G |        c |
| ----------- | -------------------------- | -------- | --------- | -------- |
| 2020-04-05  | 2020-05-01                 | 1.3977e5 | 1.7445e-1 | 1.1952e1 |
| 2020-04-06  | 2020-04-30                 | 1.2457e5 | 1.8018e-1 | 1.2271e1 |
| 2020-04-07  | 2020-04-28                 | 1.3023e5 | 1.8593e-1 | 1.2604e1 |
| 2020-04-08  | 2020-04-29                 | 1.3299e5 | 1.8192e-1 | 1.2368e1 |
| 2020-04-09  | 2020-05-02                 | 1.4172e5 | 1.7002e-1 | 1.1666e1 |
| 2020-04-10  | 2020-05-04                 | 1.4755e5 | 1.6283e-1 | 1.2344e1 |
| 2020-04-11  | 2020-05-04                 | 1.5004e5 | 1.5988e-1 | 1.1069e1 |
| 2020-04-12  | 2020-05-04                 | 1.4892e5 | 1.6125e-1 | 1.1151e1 |
| 2020-04-13  | 2020-05-04                 | 1.5011e5 | 1.5954e-1 | 4.4998e1 |
| 2020-04-15 | 2020-05-04 | 1.477e+05 | 0.1629 | 4.732 |
| 2020-04-16 | 2020-05-05 | 1.507e+05 | 0.1586 | 4.638 |
| 2020-04-17 | 2020-05-06 | 1.53e+05 | 0.1554 | 4.567 |
| 2020-04-25 | 2020-05-09 | 1.63e+05 | 0.1417 | 4.255 |
| 2020-04-26 | 2020-05-10 | 1.649e+05 | 0.1391 | 4.192 |

In [ ]:
assert isinstance(popt, LogitFitParams)
params = [datetime.now(), date100, popt.G, popt.b, popt.c]
print(markdown_table_fmt(params))

## Fitting SIR to the JHU or BMP data of Germany

Formulas are based on
* https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology

```
Susceptible (S) -> Infectious (I) -> Recovered (R)

S + I + R = N
```
Where:
* Susceptible: _"Anteil der Anfälligen / noch nicht Infizierten"_
* Infectious: _"Anteil der Infektiösen"_
* Recovered: _"Anteil der Erholten (nach Quarantäne) bzw. Verstorbenen"_



In the following we assume that all people are recovered after a duration of `SIR_INFECTIOUS_DURATION`, that is
```python
recovered = total_confirmed[-SIR_INFECTIOUS_DURATION:]
```

#### TODO:
- [ ] add SIR derivative equation in markdown cell
- [ ] plot covariance
- [ ] all constants shall have a prefix, calculated constants shall start lowercase
- [ ] also version control the output, try (binder)[https://mybinder.org/]
- [ ] In optimization usually one tries to have all the parameters in the same value range.
  Otherwise, the convergence behaviour of the optimizer might differ due to the differing scales.
  A way around this is to use a normalized form of the SIR are shown
  [here](https://www.maa.org/press/periodicals/loci/joma/the-sir-model-for-spread-of-disease-the-differential-equation-model)
- [ ] is it necessary to also estimate SIR_FIT_START_DATE with `op.curve_fit`
- [ ] what happens for differing SIR_FIT_START_DATE
- [ ] shouldn't we take gamma as constant, based on the 3 days most infective phase?
- [ ] better understand the L1 norm for L-BFGS-B and BMP data?

In [ ]:
%reset_selective -f ([xy][dipf][A-z]+[0-9]*|.*recovered.*|.*100)
%reset_selective -f (p0|pcov|popt|mask)
# NOTE: ""%reset_selective -f i" clears als datetime, unkown why
#%whos

In [ ]:
def run_SIR(t, S0, I0, R0, beta, gamma):
    def SIR(y, t, beta, gamma):
            S, I, R = y
            N = S + I + R
            dSdt = - beta * I * S / N
            dIdt = beta * I * S / N - gamma * I
            dRdt = gamma * I
            return (dSdt, dIdt, dRdt)

    return odeint(SIR, (S0, I0, R0), t, args=(beta, gamma))

`bfgs_curve_fit` is my own `curve_fit` algorithm based on the "Limited memory Broyden-Fletcher-Goldfarb-Shannon with bound constraints" minimizer / optimizer
* [scipy.optimize.minimize](http://scipy.github.io/devdocs/generated/scipy.optimize.minimize.html#scipy.optimize.minimize)
* [scipy.optimize.minimize-lbfgsb](http://scipy.github.io/devdocs/optimize.minimize-lbfgsb.html)
* [scipy.optimize.OptimizeResult](http://scipy.github.io/devdocs/generated/scipy.optimize.OptimizeResult.html#scipy.optimize.OptimizeResult)

In [ ]:
L1 = lambda x: (abs(x)).mean()
L2 = lambda x: np.sqrt((x**2).mean())

def bfgs_curve_fit(f, xdata, ydata, p0, lagrange=1.0, norm=L2, bounds=None, verbose=True, tol=1e-13):
    def bfgs_func(x, lagrange=1.0, norm=L2, ncols=2):
        ir = f(xdata, *x)
        ir = ir.reshape(-1, ncols)
        yrdata = ydata.reshape(-1, ncols)
        term = [norm(ir[:, nc] - yrdata[:, nc]) for nc in range(ncols)]
        assert ncols == 2
        return term[0] + lagrange * term[1]
    
    # ftol=1e-13, gtol=1e-13, eps=1e-7
    res = op.minimize(bfgs_func, p0, method='L-BFGS-B', args=(lagrange, norm),
                      bounds=bounds,
                      tol=tol, options=dict(disp=True))
    if verbose:
        print(res)
        min_val = bfgs_func(p0, lagrange=lagrange, norm=norm)
        print('  min_val:', min_val)
    return res['x'], res['hess_inv']

In [ ]:
if False:
    SIR_FIT_COUNTRY_KEY = 'Germany_JHU'
    SIR_FIT_RECOVERED_KEY = None
else:
    SIR_FIT_COUNTRY_KEY = 'Germany_confirmed_BMP'
    SIR_FIT_RECOVERED_KEY = 'Germany_recovered_BMP'

SIR_FIT_START_DATE = datetime(2020, 3, 2)   # repr idx 40
SIR_INFECTIOUS_DURATION = timedelta(days=4)

mask = df.index >= SIR_FIT_START_DATE
infectious_integrated = df.loc[mask, SIR_FIT_COUNTRY_KEY].values.astype('f8')
infectious = np.diff(infectious_integrated)

if SIR_FIT_RECOVERED_KEY is not None:
    recovered = df.loc[mask, SIR_FIT_RECOVERED_KEY].values.astype('f8')
    assert (not pd.isnull(recovered).any())
else:
    mask_recovered = (
        (df.index >= SIR_FIT_START_DATE - SIR_INFECTIOUS_DURATION) &
        (df.index <= df.index[-1] - SIR_INFECTIOUS_DURATION)
    )
    recovered = df.loc[mask_recovered, SIR_FIT_COUNTRY_KEY].values.astype('f8')
recovered_derivative = np.diff(recovered)

ydata = np.vstack([infectious, recovered_derivative]).transpose().astype('f8')
xdata = np.arange(0, len(ydata), dtype='f8')
print(xdata.shape, ydata.shape)

# normalize
SIR_YDATA_SCALER = 1e3
ydata_rescaled = ydata / SIR_YDATA_SCALER

ydata_rescaled.shape, ydata_rescaled.max(axis=0), SIR_YDATA_SCALER

In [ ]:
SIRFitParams = namedtuple('SIRFitParams', 'S0 I0 beta gamma'.split())

def func(t, S0, I0, beta, gamma):
    kwargs = dict(t=t,
        S0=S0, I0=I0, R0=SIR_R0,
        beta=beta, gamma=gamma,
    )
    ir = run_SIR(**kwargs)[:, 1:]
    ir[1:, 1] = np.diff(ir[:, 1])
    ir[0, 1] = 0
    return ir.ravel()

SIR_R0 = ydata_rescaled[0, 1]
p0 = SIRFitParams(
    S0=GERMANY_INHABITANTS / SIR_YDATA_SCALER - ydata_rescaled[0, 1] - ydata_rescaled[0, 0],
    I0=ydata_rescaled[0, 0],
    beta=1.2,
    gamma=1/SIR_INFECTIOUS_DURATION.days
)
# overwrite with closer solution
p0 = p0._replace(S0=500, I0=0.18, beta=1.1, gamma=0.95)

if False:
    _popt, pcov = op.curve_fit(func, xdata, ydata_rescaled.ravel(), p0)
    popt = SIRFitParams(*_popt) 
    lagrange=None
else:
    # the recovered data are rough estimates, thus we "trust" them with just 5%
    lagrange = 1.5
    _popt, pcov = bfgs_curve_fit(func, xdata, ydata_rescaled.ravel(), p0,
                                 lagrange=lagrange, norm=L2, tol=1e-13, # tol=1e-13 for L1 norm
                                 bounds=[(0, 1000), (0, 10), (0.01, 2), (0.05, 10)])
    popt = SIRFitParams(*_popt) 

print(p0)
print(popt)

infectious_duration = 1 / popt.gamma
print('Estimated infection duration:', infectious_duration)

# do not confuse reproduction number R_0 (R subscript 0) with
# the recovered cases at time zero R0
R_0 = popt.beta / popt.gamma
print('Estimated reproduction number R_0 = %.2f' % R_0)

In [ ]:
logit_days_to_predict = 90
xpredict = np.arange(len(xdata) + logit_days_to_predict)
sol = run_SIR(t=xpredict, R0=SIR_R0, **popt._asdict())

fig, ax = plt.subplots(2, 1, sharex=True, figsize=MPL_FIG_SIZE_LARGE, constrained_layout=True)

ax[0].plot(xpredict, sol[:, 1], label='Infectious', color=FIG_COLOR_PALETTE[1])
ax[0].plot(xpredict[1:], np.diff(sol[:, 2]), label='diff(Recovered)', color=FIG_COLOR_PALETTE[5], alpha=0.3)
ax[0].scatter(xdata, ydata_rescaled[:, 0], color=FIG_COLOR_PALETTE[1])
ax[0].scatter(xdata, ydata_rescaled[:, 1], color=FIG_COLOR_PALETTE[5], alpha=0.3)
ax[0].set_ylabel('cases scaled by %d' % SIR_YDATA_SCALER)

IDX_OFFSET_TODAY = len(ydata)
idx100 = np.argmin(abs(sol[IDX_OFFSET_TODAY:, 1] - 100/SIR_YDATA_SCALER)) + IDX_OFFSET_TODAY
date100 = SIR_FIT_START_DATE + timedelta(days=int(idx100))
ax[0].axvline(idx100, color='0.6', ls='--', label='100 new cases @ %s' % date100.strftime(MPL_FIG_TITLE_DATE_FORMAT))
ax[0].plot([], [], ' ', label='Estimated $R_0$ = %.2f' % R_0) # dirty way to add legend

for i, name in enumerate(['Susceptible', 'Infectious', 'Recovered']):
    ax[1].plot(xpredict, sol[:, i] * SIR_YDATA_SCALER, label=name)
ax[1].scatter(xdata, ydata_rescaled[:, 0] * SIR_YDATA_SCALER, color=FIG_COLOR_PALETTE[1])
ax[1].scatter(xdata, recovered[1:] , color=FIG_COLOR_PALETTE[2])
ax[1].legend(loc='best')
ax[1].set_xlabel('time [days]')
ax[1].set_ylabel('cases unscaled')

for i in [0, 1]:
    ax[i].legend().get_frame().set_alpha(0.9)

### Date of 100 new infections and reproduction number Germany based on SIR fit

| fit @ | $I = 100$ @ | $R_0$ | infectious days | S0 | I0 | SIR_R0 | beta | gamma | SIR_YDATA_SCALER | key | fit fct | lagrange |
|:-----:| ------------| ----- | --------------- | -- | -- | ------ | ---- | ----- | ---------------- | --- | ------- | -------- |
| 2020-04-13 | 2020-05-06  | 1.175 | 1.048 | 505.9 | 0.1759 | 0.002 | 1.121 | 0.9542 | 1000 | JHU | op.curve_fit | - |
|   - " -    | 2020-05-11  | 1.3 | 1.88 | 197.8 | 0.2029 | 0 | 0.6914 | 0.532 | 1000 | BMP | op.curve_fit | - |
|   - " -    | 2020-05-07 | 1.383 | 2.222 | 136.5 | 0.2014 | 0 | 0.6223 | 0.4501 | 1000 | BMP | bfgs_curve_fit | 0.05 |
| 2020-04-15 | 2020-05-06 | 1.354 | 2.027 | 154.3 | 0.1914 | 0 | 0.668 | 0.4932 | 1000 | BMP | bfgs_curve_fit | 0.05 |
| 2020-04-16 | 2020-05-06 | 1.345 | 1.985 | 160.4 | 0.1937 | 0 | 0.678 | 0.5039 | 1000 | BMP | bfgs_curve_fit | 0.05 |
| 2020-04-17 | 2020-05-07 | 1.349 | 2.051 | 156.5 | 0.2082 | 0 | 0.6578 | 0.4875 | 1000 | BMP | bfgs_curve_fit | 0.05 |
| 2020-04-25 | 2020-05-21 | 1.171 | 1.388 | 439.2 | 0.3178 | 0 | 0.8434 | 0.7203 | 1000 | BMP | bfgs_curve_fit | 1.5 |
| 2020-04-26 | 2020-05-24 | 1.152 | 1.324 | 517.7 | 0.3429 | 0 | 0.8704 | 0.7553 | 1000 | BMP | bfgs_curve_fit | 1.5 |


In [ ]:
assert isinstance(popt, SIRFitParams)
params = [datetime.now(), date100, R_0, infectious_duration,
          popt.S0, popt.I0, SIR_R0, popt.beta, popt.gamma, SIR_YDATA_SCALER, '???', '???curve_fit', lagrange]
print(markdown_table_fmt(params))